In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D, Lambda, MaxPooling2D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Conv2D, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.utils import to_categorical
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [2]:
df=pd.read_csv('Twitter_3/data_train.csv',header=None)
df = df.iloc[1:]
df_test=pd.read_csv('Twitter_3/data_test.csv',header=None)
df_test = df_test.iloc[1:]
X_train=df.iloc[:,0]
Y_tr=df.iloc[:,2]
X_test=df_test.iloc[:,0]
Y_t=df_test.iloc[:,2]


FileNotFoundError: [Errno 2] No such file or directory: 'Twitter_3/data_train.csv'

Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [4]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [9]:
embedding_dim=300
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    branch1 = Sequential()
    branch1.add(Embedding(max_word,embedding_dim,input_length=max_len))
    branch1.add(Conv1D(64,3,padding='same',activation='relu'))
    branch1.add(BatchNormalization())
    branch1.add(ReLU())
    branch1.add(Dropout(0.5))
    branch1.add(GlobalMaxPooling1D())
    input1 = Input(shape=(max_len,))
    x1=branch1(input1)

    hid_layer=Dense(128,activation='relu')(x1)
    drop_layer=Dropout(0.3)(hid_layer)
    outp=Dense(3, activation='softmax')(drop_layer)
    model=Model(inputs=input1,outputs=outp)
    model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy',Precision(),Recall()])
    return model

Chạy mô hình với data train

In [10]:
model=model1()
model.summary()


c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 64)             │     6,057,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,066,627 (23.14 MB)

 Trainable params: 6,066,499 (23.14 MB)

 Non-trainable params: 128 (512.00 B)

In [11]:
history=model.fit(tr_x, tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

Epoch 1/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.4165 - loss: 1.2420 - precision_1: 0.4404 - recall_1: 0.2886 - val_accuracy: 0.5843 - val_loss: 1.0392 - val_precision_1: 1.0000 - val_recall_1: 0.0018
Epoch 2/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.6101 - loss: 0.8519 - precision_1: 0.6684 - recall_1: 0.4705 - val_accuracy: 0.6678 - val_loss: 0.8400 - val_precision_1: 0.8469 - val_recall_1: 0.2978
Epoch 3/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.6865 - loss: 0.7297 - precision_1: 0.7302 - recall_1: 0.6074 - val_accuracy: 0.6896 - val_loss: 0.7584 - val_precision_1: 0.8034 - val_recall_1: 0.4646
Epoch 4/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.7350 - loss: 0.6343 - precision_1: 0.7686 - recall_1: 0.6815 - val_accuracy: 0.7086 - val_loss: 0.7199 - val_precision_1: 0.8007 - val_recall_1: 0.5328
Epoch 5/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.7753 - loss: 0.5635 - precision_1: 0.7995 - recall_1

Thử dữ liệu với data test

In [12]:
pred_label=model.predict(t_x)
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))


111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.65      0.69      0.67      1001
           1       0.63      0.66      0.64      1430
           2       0.77      0.69      0.73      1103

    accuracy                           0.68      3534
   macro avg       0.68      0.68      0.68      3534
weighted avg       0.68      0.68      0.68      3534

